## Make a drug response matrix that allows people to use drug names in their consideration
Built from Supplementary Table 2

In [5]:
import pandas as pd
import numpy as np

# Convert Supplementary Table 2 into a drug response matrix, including the train/test split
drug_all = pd.read_csv('drug_response_SuppTable2.txt', sep='\t')

drug_all.head()

drug_all.head()

# get total unique compounds and cell lines
unique_compounds = drug_all['compound'].unique()
unique_cell_lines = drug_all['cell_line_name'].unique()

print(f"Total unique compounds: {len(unique_compounds)}")
print(f"Total unique cell lines: {len(unique_cell_lines)}")


# create empty dataframe
drug_response_full = pd.DataFrame(index=unique_cell_lines, columns=unique_compounds)
drug_response_train = pd.DataFrame()
drug_response_test = pd.DataFrame()
# fill in the dataframe
for _, row in drug_all.iterrows():
    # gi50 in is the -1log10 of the concentration required to inhibit growth by 50%.
    drug_response_full.at[row['cell_line_name'], row['compound']] = -1 * np.log10(row['gi50'])
    if row['test.train'] == 'Test':
        drug_response_test.at[row['cell_line_name'], row['compound']] = -1 * np.log10(row['gi50'])

    else:
        drug_response_train.at[row['cell_line_name'], row['compound']] = -1 * np.log10(row['gi50'])

print("Full drug response matrix shape:", drug_response_full.shape)
print("Train drug response matrix shape:", drug_response_train.shape)
print("Test drug response matrix shape:", drug_response_test.shape)# make a full drug set, this uses the compund name as the column headers and the gi50 values as the data
#drug_response_full = drug_all.pivot(index='cell_line_name', columns='compound', values='gi50')



# save these datasets
drug_response_full.to_csv('drug_response_full_matrix.csv')
drug_response_train.to_csv('drug_response_train_matrix.csv')
drug_response_test.to_csv('drug_response_test_matrix.csv')

Total unique compounds: 28
Total unique cell lines: 53
Full drug response matrix shape: (53, 28)
Train drug response matrix shape: (35, 28)
Test drug response matrix shape: (18, 28)


In [ ]:
# do these data make sense? compare to the prediction set
predictions = pd.read_csv('DREAM7_DrugSensitivity1_Predictions.csv', index_col=0)
print("Predictions shape:", predictions.shape)

# that is more drugs than what I have in SuppTable 2, let's check SuppTable10

Predictions shape: (53, 31)


## Get all unique gene names and cell names for adding infromation for metadata

In [43]:


# DRUG RESPONSE DATA in the form of -1log10(GI50) values. The GI50 is the concentration of drug needed to inhibit growth of the cell line by 50%.
# taking the -1log10 makes higher values better response (lower concentration needed). a value of 6 means 1 micromolar concentration needed, 5 means 10 micromolar, etc.
drug_response_full = pd.read_csv('drug_response_full_matrix.csv', index_col=0)
drug_response_train = pd.read_csv('drug_response_train_matrix.csv', index_col=0)
drug_response_test = pd.read_csv('drug_response_test_matrix.csv', index_col=0)
drug_response_classification = pd.read_csv('drug_response_classification.txt', sep='\t', index_col=0)
print("Data shapes:")
print('Full drug response data shape:', drug_response_full.shape)
print('Training drug response data shape:', drug_response_train.shape)
print('Test drug response data shape:', drug_response_test.shape)
print('Classification drug response data shape:', drug_response_classification.shape)


# Load the molecular data, all molecules on the gene level will have gene names as rows and cell lines as columns

# GENE EXPRESSION and GENE-level data

# There's gene expression data in the form of RANA-seq quantification and microarrays
RNA_seq_quant = pd.read_csv('RNAseq_quantification.txt', sep='\t', index_col=0)
RNA_seq_calls = pd.read_csv('RNAseq_expressed_calls.txt', sep='\t', index_col=0)
mRNA = pd.read_csv('GeneExpression.txt' , sep='\t', index_col=0)

# There is gene-level copy number variation data
copy_number = pd.read_csv('SNP6_gene_level.txt', sep='\t', index_col=0)

# Methylation data exists too
methylation = pd.read_csv('Methylation.txt', sep='\t', index_col=0)

# There's also information about mutations in genes, in exome sequencing data 
exome_seq = pd.read_csv('ExomeSeq.txt', sep='\t', index_col=0)

# PROTEIN-LEVEL DATA
# There is protein expression data from Reverse Phase Protein Arrays (RPPA)
rppa = pd.read_csv('RPPA.txt', sep='\t', index_col=0)
print("Data shapes:")
print('RNA-seq quantification data shape:', RNA_seq_quant.shape)
print('RNA-seq expressed calls data shape:', RNA_seq_calls.shape)
print('mRNA gene expression data shape:', mRNA.shape)
print('Copy number variation data shape:', copy_number.shape)
print('Methylation data shape:', methylation.shape) 
print('Exome sequencing data shape:', exome_seq.shape)  
print('RPPA protein expression data shape:', rppa.shape)

Data shapes:
Full drug response data shape: (53, 28)
Training drug response data shape: (35, 28)
Test drug response data shape: (18, 28)
Classification drug response data shape: (53, 28)
Data shapes:
RNA-seq quantification data shape: (36953, 45)
RNA-seq expressed calls data shape: (36953, 45)
mRNA gene expression data shape: (18632, 46)
Copy number variation data shape: (27234, 48)
Methylation data shape: (27551, 44)
Exome sequencing data shape: (33160, 26)
RPPA protein expression data shape: (131, 43)


In [69]:
# The full set of cell lines and the intersection between datasets
datasets = {}
#datasets['rppa'] = rppa
datasets['mrna'] = mRNA#set_index('HGNC_ID')
datasets['rnaseq'] = RNA_seq_quant#.set_index('HGNC_ID')

#set copy_number index to the HGNC_ID so they can be unioned by the same gene name
datasets['copy'] = copy_number.set_index('HGNC_ID')
datasets['methylation'] = methylation.set_index('HGNC_ID')

unique_cell_lines = []
unique_gene_names = []
for data_name in datasets.keys():
    gene_names = datasets[data_name].index.to_list()
    cell_names = datasets[data_name].columns.to_list()
    # union the new cell_names with the ongoing list and expand as necessary
    unique_cell_lines = list(set.union(set(cell_names), set(unique_cell_lines)))
    unique_gene_names = list(set.union(set(gene_names), set(unique_gene_names)))
    

In [73]:
print(unique_gene_names)

['LOC645993', 'NCRNA00081', 'PRKAG3', 'SCARNA16', 'LOC388276', 'ZNF577', 'GRLF1', 'AC011998.8', 'AL590705.9', 'SLAMF1', 'RP11-433K2.1', 'HMGB4', 'AC084114.8', 'DRD5P2', 'RBP4', 'RP11-291J9.2', 'ZNF451', 'BX276116.7', 'SERTAD1', 'LOC646667', 'LOC390285', 'RP11-143M1.1', 'LOC645064', 'CECR2', 'LOC643436', 'REP15', 'LOC645511', 'LOC646486', 'hsa-mir-30c-1', 'CES7', 'LGALS3BP', 'P8', 'LOC286042', 'FLJ25067', 'ALAS2', 'GLIPR1L1', 'VSTM1', 'hsa-mir-1470', 'LOC644766', 'LOC644865', 'hsa-mir-518e', 'LOC144097', 'RP4-742C19.11', 'CHORDC2P', 'UBR2', 'C2orf7', 'POU6F1', 'hsa-mir-196a-2', 'OR13D1', 'CRYBB3', 'LOC338097', 'SCAP1', 'DDA1', 'LOC645167', 'RP13-34L8.3', 'AC104133.5', 'ZACN', 'C13orf30', 'C20orf44', 'AC007318.4', 'LOC643528', 'hsa-mir-619', 'ZNF673', 'AL590704.13', 'RP11-810H18.1', 'LOC402122', 'LOC647006', 'DSPP', 'LOC653472', 'BTC', 'CDKL4', 'ISOC1', 'RQCD1', 'BX664726.3', 'COX5B', 'C14orf29', 'MDS2', 'VPS33A', 'LA16c-2F2.7', 'OR8B10P', 'CERK', 'MORF4', 'ABCD1P4', 'SIX6', 'CXorf30', '

In [70]:
print("unique cell lines: %d"%len(unique_cell_lines))
print("unique genes: %d"%len(unique_gene_names))

unique cell lines: 55
unique genes: 42995


In [15]:
# let's check the cell line information
cell_line_info = pd.read_csv('cell_line_info.csv')
cell_line_info
cell_lines = cell_line_info['Cell Line Name'].to_list()

overlap = set(cell_lines).intersection(set(unique_cell_lines))
missing = set(unique_cell_lines) - set(cell_lines)

print("Missing cell lines from molecular data:", missing)
print("Overlapping cell lines:", overlap)

Missing cell lines from molecular data: {'184B5', 'HCC1806', 'HCC1395', 'MCF10F', 'MX1', 'HCC1419', '184A1', '21MT1', 'SUM229PE', 'T47DKBLUC', '21NT'}
Overlapping cell lines: {'HCC1187', 'MDAMB415', '600MPE', 'ZR75B', 'CAMA1', 'UACC812', 'BT483', 'MDAMB134VI', 'T47D', 'BT474', 'BT549', 'ZR751', 'MDAMB175VII', 'ZR7530', 'HCC1569', 'HCC1937', 'MDAMB157', 'SUM159PT', 'SUM225CWN', 'SKBR3', 'HCC70', 'SUM1315MO2', 'HCC1143', 'HCC3153', 'MDAMB231', 'HCC2185', 'MDAMB361', 'MDAMB453', 'SUM52PE', 'MCF10A', 'HCC1954', 'HCC38', 'AU565', 'MCF12A', 'SUM149PT', 'HCC202', 'HCC1428', 'SUM185PE', 'BT20', 'MCF7', 'LY2', 'HS578T'}


In [16]:
cell_line_info.columns

Index(['Cell line', 'Gene cluster', 'Cell Line Name', 'ER', 'PR', 'HER2',
       'TP53 allele 1 ', 'TP53 allele 2', 'Source', 'Tumor type',
       'Age (years)', 'Ethnicity', 'Culture media', 'Culture conditions'],
      dtype='object')

In [ ]:
# build a gene set information 
# Here's where to download gene sets from MSigDB: https://www.gsea-msigdb.org/gsea/msigdb/collections.jsp
# I pulled the hallmarks as a starter.
msig_hallmarks_file = 'h.all.v2025.1.Hs.symbols.gmt'
msig_hallmarks = {}
with open(msig_hallmarks_file, 'r') as f:
    for line in f:
        parts = line.strip().split('\t')
        gene_set_name = parts[0]
        gene_set_genes = parts[2:]  # skip the first two columns (name and description)
        msig_hallmarks[gene_set_name] = gene_set_genes

genes_msig_annotated = pd.DataFrame(index=unique_gene_names)
for gene_set_name, gene_set_genes in msig_hallmarks.items():
    genes_msig_annotated[gene_set_name] = genes_msig_annotated.index.isin(gene_set_genes).astype(int)

In [79]:
genes_msig_annotated.sum()
genes_msig_annotated.to_csv('genes_msig_hallmarks_annotation.csv')

In [ ]:
# how about information about the cell lines

{'HALLMARK_ADIPOGENESIS': ['ABCA1',
  'ABCB8',
  'ACAA2',
  'ACADL',
  'ACADM',
  'ACADS',
  'ACLY',
  'ACO2',
  'ACOX1',
  'ADCY6',
  'ADIG',
  'ADIPOQ',
  'ADIPOR2',
  'AGPAT3',
  'AIFM1',
  'AK2',
  'ALDH2',
  'ALDOA',
  'ANGPT1',
  'ANGPTL4',
  'APLP2',
  'APOE',
  'ARAF',
  'ARL4A',
  'ATL2',
  'ATP1B3',
  'ATP5PO',
  'BAZ2A',
  'BCKDHA',
  'BCL2L13',
  'BCL6',
  'C3',
  'CAT',
  'CAVIN1',
  'CAVIN2',
  'CCNG2',
  'CD151',
  'CD302',
  'CD36',
  'CDKN2C',
  'CHCHD10',
  'CHUK',
  'CIDEA',
  'CMBL',
  'CMPK1',
  'COL15A1',
  'COL4A1',
  'COQ3',
  'COQ5',
  'COQ9',
  'COX6A1',
  'COX7B',
  'COX8A',
  'CPT2',
  'CRAT',
  'CS',
  'CYC1',
  'CYP4B1',
  'DBT',
  'DDT',
  'DECR1',
  'DGAT1',
  'DHCR7',
  'DHRS7',
  'DHRS7B',
  'DLAT',
  'DLD',
  'DNAJB9',
  'DNAJC15',
  'DRAM2',
  'ECH1',
  'ECHS1',
  'ELMOD3',
  'ELOVL6',
  'ENPP2',
  'EPHX2',
  'ESRRA',
  'ESYT1',
  'ETFB',
  'FABP4',
  'FAH',
  'FZD4',
  'G3BP2',
  'GADD45A',
  'GBE1',
  'GHITM',
  'GPAM',
  'GPAT4',
  'GPD2',
  'GPHN

In [32]:
suppTable10 = pd.read_csv('drug_response_SuppTable10.txt', sep='\t', header=[0, 1]) # it has a double header row
suppTable10.head()

,Cell line,Cetuximab (EGFR),Unnamed: 2_level_0,Unnamed: 3_level_0,Trastuzumab (ERBB2),Unnamed: 5_level_0,Unnamed: 6_level_0,PS-1145 (IKK),Unnamed: 8_level_0,Unnamed: 9_level_0,...,Unnamed: 75_level_0,4-HC (DNA alkylator),Unnamed: 77_level_0,Unnamed: 78_level_0,4-HC+Dox (Combination),Unnamed: 80_level_0,Unnamed: 81_level_0,Doxorubicin (TOP2A),Unnamed: 83_level_0,Unnamed: 84_level_0
,Unnamed: 0_level_1,GI50,replicates,SD.geom,GI50,replicates,SD.geom,GI50,replicates,SD.geom,...,SD.geom,GI50,replicates,SD.geom,GI50,replicates,SD.geom,GI50,replicates,SD.geom
0,184A1,7.216382,2.0,1.487831,5.573489,1.0,NaN,2.723538,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,184B5,7.609065,1.0,NaN,5.142668,1.0,NaN,2.723538,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21MT1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.172019,4.607303,1.0,NaN,4.690370,1.0,NaN,6.614394,1.0,NaN
3,21NT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.110926,5.202940,2.0,0.393926,5.108518,2.0,0.195069,6.741123,2.0,0.001734
4,600MPE,6.164309,2.0,0.000000,NaN,NaN,NaN,2.723538,2.0,0.0,...,NaN,4.488117,1.0,NaN,4.583359,1.0,NaN,6.663540,1.0,NaN


In [ ]:
suppTable10
suppTable10_gi50 = suppTable10.copy()
# let's keep Cell line as the index and then keep the GI50 columns only, switching column name to the row value of header 0
suppTable10_gi50 = suppTable10_gi50.xs('GI50', axis=1, level=1)  # select only GI50 columns 
suppTable10_gi50.head()

# also only 28 drugs in this, so let's assume we just have 28 drugs, I don't know where the missing drugs are.

,Cetuximab (EGFR),Trastuzumab (ERBB2),PS-1145 (IKK),Baicalein (CYP2C9),GW5074 (RAF1),IKK 16 (IKK2),Imatinib (BCR-ABL),QNZ (NFkB),FR180304 (ERK),Everolimus (mTOR),...,Disulfiram (ALDH2),Mebendazole (Tubulin),Chloroquine (Autophagy),Bromopyruvate (Glycolysis),Methylglyoxol (Pyruvate),MG-132b (Proteasome),MG-115 (Proteasome),4-HC (DNA alkylator),4-HC+Dox (Combination),Doxorubicin (TOP2A)
0,7.216382,5.573489,2.723538,3.777284,5.694649,6.045757,4.860121,4.477556,4.605548,6.164944,...,4.899629,4.175874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7.609065,5.142668,2.723538,3.777284,5.521434,5.826814,4.625435,7.462181,4.889410,5.549751,...,4.931814,4.175874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.175874,4.742321,NaN,NaN,3.074172,7.001479,6.429691,4.607303,4.690370,6.614394
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.718967,6.075721,NaN,NaN,3.271646,7.018816,6.587036,5.202940,5.108518,6.741123
4,6.164309,NaN,2.723538,3.777284,4.477556,5.110698,4.622512,4.477556,4.649752,4.701147,...,7.095826,5.002177,7.917215,7.497573,3.216096,6.309804,5.886057,4.488117,4.583359,6.663540


In [ ]:
# Drug classification 